In [15]:
# Download dataset
import requests
url = 'https://www.openml.org/data/get_csv/16826755/phpMYEkMl'
filename = 'dataset.csv'
r = requests.get(url, allow_redirects=True)
open(filename, 'wb').write(r.content)

117743